<a href="https://colab.research.google.com/github/rohanjain1648/TECHNOVA-HACKATHON/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np

# Generating a sample dataset
data = {
    'past_transactions_1d': np.random.randint(1, 10, 1000),
    'past_transactions_1w': np.random.randint(5, 50, 1000),
    'avg_transaction_amount': np.random.uniform(5, 200, 1000),
    'highest_transaction_amount': np.random.uniform(100, 5000, 1000),
    'transaction_location': np.random.choice(['City A', 'City B', 'City C'], 1000),
    'transaction_type': np.random.choice(['Online Purchase', 'ATM Withdrawal', 'Cash Deposit'], 1000),
    'current_transaction_amount': np.random.uniform(10, 500, 1000),
    'current_transaction_location': np.random.choice(['City A', 'City B', 'City C'], 1000),
    'transaction_time': pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", periods=1000, freq="H"), 1000)),
    'merchant_category': np.random.choice(['Retail', 'Food', 'Electronics'], 1000),
    'device_used': np.random.choice(['Mobile', 'PC'], 1000),
    'IP_address': np.random.choice(['192.168.0.1', '10.0.0.1', '172.16.0.1'], 1000),
    'account_age': np.random.randint(1, 20, 1000),
    'customer_loyalty_level': np.random.choice(['Bronze', 'Silver', 'Gold'], 1000),
    'billing_address': np.random.choice(['Address 1', 'Address 2', 'Address 3'], 1000),
    'shipping_address': np.random.choice(['Address 1', 'Address 2', 'Address 3'], 1000),
    'fraud': np.random.choice([0, 1], 1000)  # Target: 0 (No Fraud), 1 (Fraud)
}

# Creating DataFrame
df = pd.DataFrame(data)

# Saving the dataset to CSV for training purposes
df.to_csv("transaction_data.csv", index=False)


<ipython-input-26-d0555de924cf>:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'transaction_time': pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", periods=1000, freq="H"), 1000)),


In [28]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("transaction_data.csv")

# Convert transaction_time to datetime and extract time-based features
df['transaction_time'] = pd.to_datetime(df['transaction_time'], errors='coerce')
df['transaction_hour'] = df['transaction_time'].dt.hour
df['transaction_day'] = df['transaction_time'].dt.dayofweek
df.drop(columns=['transaction_time'], inplace=True)

# Encode categorical variables using LabelEncoder
categorical_cols = ['transaction_location', 'transaction_type', 'current_transaction_location',
                    'merchant_category', 'device_used', 'IP_address', 'customer_loyalty_level',
                    'billing_address', 'shipping_address']

encoder_dict = {}  # Dictionary to store encoders for later use
for col in categorical_cols:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    encoder_dict[col] = encoder  # Save encoder for manual input

# Define features & labels
X = df.drop(columns=['fraud'])
y = df['fraud']

# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
xgb_model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


# -------------- MANUAL INPUT FUNCTION --------------
def manual_input_prediction(model, X_columns, encoder_dict):
    print("\nEnter transaction details:")

    # Collect user input
    user_input = {
        'past_transactions_1d': int(input("Past transactions in last day: ")),
        'past_transactions_1w': int(input("Past transactions in last week: ")),
        'avg_transaction_amount': float(input("Average transaction amount: ")),
        'highest_transaction_amount': float(input("Highest transaction amount: ")),
        'transaction_location': input("Transaction location (City A/City B/City C): "),
        'transaction_type': input("Transaction type (Online Purchase/ATM Withdrawal/Cash Deposit): "),
        'current_transaction_amount': float(input("Current transaction amount: ")),
        'current_transaction_location': input("Current transaction location (City A/City B/City C): "),
        'merchant_category': input("Merchant category (Retail/Food/Electronics): "),
        'device_used': input("Device used (Mobile/PC): "),
        'IP_address': input("IP address (192.168.0.1/10.0.0.1/172.16.0.1): "),
        'account_age': int(input("Account age (in years): ")),
        'customer_loyalty_level': input("Customer loyalty level (Bronze/Silver/Gold): "),
        'billing_address': input("Billing address (Address 1/Address 2/Address 3): "),
        'shipping_address': input("Shipping address (Address 1/Address 2/Address 3): "),
        'transaction_hour': int(input("Transaction hour (0-23): ")),
        'transaction_day': int(input("Transaction day of week (0=Monday, 6=Sunday): "))
    }

    # Convert user input to DataFrame
    user_df = pd.DataFrame([user_input])

    # Encode categorical variables using previously fitted encoders
    for col in categorical_cols:
        if col in encoder_dict:
            user_df[col] = encoder_dict[col].transform([user_df[col][0]])  # Encode single value

    # Ensure column order matches training data
    user_df = user_df[X_columns]

    # Make prediction
    prediction = model.predict(user_df)[0]

    # Display result
    if prediction == 1:
        print("🚨 Warning: Potential Fraud Detected!")
    else:
        print("✅ Transaction appears safe.")

# Call the function
manual_input_prediction(xgb_model, X.columns, encoder_dict)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:52:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Enter transaction details:
Past transactions in last day: 50
Past transactions in last week: 300
Average transaction amount: 5000
Highest transaction amount: 20000
Transaction location (City A/City B/City C): City B
Transaction type (Online Purchase/ATM Withdrawal/Cash Deposit): ATM Withdrawal
Current transaction amount: 15000
Current transaction location (City A/City B/City C): City C
Merchant category (Retail/Food/Electronics): Electronics
Device used (Mobile/PC): PC
IP address (192.168.0.1/10.0.0.1/172.16.0.1): 10.0.0.1
Account age (in years): 1
Customer loyalty level (Bronze/Silver/Gold): Bronze
Billing address (Address 1/Address 2/Address 3): Address 3
Shipping address (Address 1/Address 2/Address 3): Address 3
Transaction hour (0-23): 2
Transaction day of week (0=Monday, 6=Sunday): 6
🚨 Warning: Potential Fraud Detected!
